### Sirviendo modelos con MLFlow.

Un caso rápido con Pytorch e Iris Dataset.

In [1]:
import torch
import pickle

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

iris = pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/Iris_people/iris.csv')

##### ¿A qué se le llaman artifacts?

Artifacts no son más que objetos útiles e instanciados que usaremos para hacer nuestro proceso de predicción. Por ejemplo, puede ser un modelo ya entrenado, un diccionario que encodee algunas variables categóricas, o el famoso state_dict (los pesos del modelo) de Pytorch. En este caso, es necesario usar un LabelEncoder para poder continuar con el proceso de nuestro modelo. 

In [2]:
L_encoder = LabelEncoder()

iris['tipo_flor'] = L_encoder.fit_transform(iris['tipo_flor'])

#Tenemos que guardar este artifact para la inferecncia posterior.

with open('artifacts/label_encoder.pkl', 'wb') as handle:
    pickle.dump(L_encoder, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [3]:
#Ahora, para entrenar generaremos un X_train y X_test con un random seed de 10.
X_train, X_test, Y_train, Y_test = train_test_split(iris.drop('tipo_flor',axis=1),
                                                    iris['tipo_flor'], random_state=10)

In [4]:
#Como usaremos Pytorch, convertiremos nuestro dataframe a tensores.
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()
y_train = torch.from_numpy(Y_train.values).view(1,-1)[0].type(torch.LongTensor)
y_test = torch.from_numpy(Y_test.values).view(1,-1)[0].type(torch.LongTensor)

#### Dónde armamos el modelo?

El modelo va a estar definido en un script aparte llamado modelo.py. Esto nos permitirá importar la clase del modelo, con toda la arquitectura de la red neuronal que hayamos decidido usar. ¿Por qué hacemos esto? Porque es una forma más formal de almacenar nuestro modelo para una posterior puesta en producción.

In [5]:
from modelo import ModeloIris
from torch.optim import Adam
import torch.nn as nn

model = ModeloIris()
#Inicializamos el modelo, seteando las configuraciones:

optimizer = Adam(model.parameters(), lr = 0.01)
fn_perdida = nn.NLLLoss()

In [6]:
#Entrenaremos el modelo.
epochs = 1000

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = fn_perdida(y_pred, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch: {epoch} loss: {loss.item()}')

Epoch: 0 loss: 1.1388798952102661
Epoch: 100 loss: 0.055816106498241425
Epoch: 200 loss: 0.0496302954852581
Epoch: 300 loss: 0.04864420369267464
Epoch: 400 loss: 0.04905252903699875
Epoch: 500 loss: 0.047640495002269745
Epoch: 600 loss: 0.04695656895637512
Epoch: 700 loss: 0.047282785177230835
Epoch: 800 loss: 0.04627826437354088
Epoch: 900 loss: 0.04583793133497238


In [7]:
import mlflow.pytorch

mlflow_pytorch_path = 'models/iris_mlflow_pytorch'

In [8]:
#Sacaremos el environment de anaconda que estamos usando:
conda_env = mlflow.pytorch.get_default_conda_env()
print(conda_env)

{'name': 'mlflow-env', 'channels': ['defaults', 'conda-forge', 'pytorch'], 'dependencies': ['python=3.8.5', 'pytorch=1.8.1+cpu', 'torchvision=0.9.1+cpu', 'pip', {'pip': ['mlflow', 'cloudpickle==1.6.0']}]}


In [10]:
#Guardaremos el modelo
mlflow.pytorch.save_model(model, mlflow_pytorch_path, conda_env = conda_env)

#### Hasta acá ya tenemos nuestro modelo. Lo que falta es poder cargarlo desde los archivos.

In [11]:
loaded_model = mlflow.pytorch.load_model(mlflow_pytorch_path)

#Tomamos un ejemplo para hacer la inferencia.

ejemplo = torch.tensor([[3.0, 2.5, 1.8, 0.8]])

prediccion = torch.argmax(loaded_model(ejemplo), dim = 1)

print(prediccion)

tensor([0])
